In [1]:
from faim_hcs.io.YokogawaCellVoyager import parse_files, parse_metadata
from faim_hcs.CellVoyagerUtils import get_well_image_CZYX
from faim_hcs.Zarr import build_zarr_scaffold, write_czyx_image_to_well, PlateLayout, write_roi_table
from os.path import join, exists
from tqdm.notebook import tqdm

import numpy as np
import shutil

In [2]:
directory = "../resources/CV8000/CV8000-Minimal-DataSet-2C-3W-4S-FP2-stack_20230918_135839/CV8000-Minimal-DataSet-2C-3W-4S-FP2-stack/"
zarr_root = './zarr-files'

In [3]:
files = parse_files(acquisition_dir=directory)

In [4]:
files

,Time,TimePoint,FieldIndex,ZIndex,TimelineIndex,ActionIndex,Action,X,Y,Z,Ch,path,well
0,2023-09-18T13:58:44.438+02:00,1,1,1,1,1,3D,-567.1,402.9,0.0,1,../resources/CV8000/CV8000-Minimal-DataSet-2C-...,D08
1,2023-09-18T13:58:44.647+02:00,1,1,2,1,1,3D,-567.1,402.9,3.0,1,../resources/CV8000/CV8000-Minimal-DataSet-2C-...,D08
2,2023-09-18T13:58:44.863+02:00,1,1,3,1,1,3D,-567.1,402.9,6.0,1,../resources/CV8000/CV8000-Minimal-DataSet-2C-...,D08
3,2023-09-18T13:58:45.080+02:00,1,1,4,1,1,3D,-567.1,402.9,9.0,1,../resources/CV8000/CV8000-Minimal-DataSet-2C-...,D08
4,2023-09-18T13:58:45.568+02:00,1,1,1,1,2,3D,-567.1,402.9,0.0,2,../resources/CV8000/CV8000-Minimal-DataSet-2C-...,D08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2023-09-18T13:59:13.797+02:00,1,4,4,1,1,3D,82.8,-247.0,9.0,1,../resources/CV8000/CV8000-Minimal-DataSet-2C-...,F08
92,2023-09-18T13:59:14.300+02:00,1,4,1,1,2,3D,82.8,-247.0,0.0,2,../resources/CV8000/CV8000-Minimal-DataSet-2C-...,F08
93,2023-09-18T13:59:14.501+02:00,1,4,2,1,2,3D,82.8,-247.0,3.0,2,../resources/CV8000/CV8000-Minimal-DataSet-2C-...,F08
94,2023-09-18T13:59:14.704+02:00,1,4,3,1,2,3D,82.8,-247.0,6.0,2,../resources/CV8000/CV8000-Minimal-DataSet-2C-...,F08


In [5]:
name, channel_metadata = parse_metadata(acquistion_dir=directory)

In [6]:
channel_metadata

,Ch,HorizontalPixelDimension,VerticalPixelDimension,CameraNumber,InputBitDepth,InputLevel_x,HorizontalPixels,VerticalPixels,FilterWheelPosition,FilterPosition,...,Binning,Color,MinLevel,MaxLevel,CSUID,PinholeDiameter,Kind,CameraType,InputLevel_y,Fluorophore
0,1,0.325,0.325,3,16,10000,2000,2000,6,1,...,1,#FF002FFF,0,0.59884668421712428,2,50,ConfocalFluorescence,DCAM,10000,
1,2,0.325,0.325,3,16,10000,2000,2000,6,2,...,1,#FF00FFA1,0,0.078600977811206979,2,50,ConfocalFluorescence,DCAM,10000,
2,3,0.325,0.325,4,16,10000,2000,2000,7,1,...,1,#FFFF8200,0,0.16008524507960364,2,50,ConfocalFluorescence,DCAM,10000,
3,4,0.325,0.325,4,16,10000,2000,2000,7,2,...,1,#FFFF1B00,0,0.078600977811207,2,50,ConfocalFluorescence,DCAM,10000,


In [7]:
files["name"] = "CV8000_sample"

In [8]:
if exists(join(zarr_root, "CV8000_sample.zarr")):
    # Remove zarr if it already exists.
    shutil.rmtree(join(zarr_root, "CV8000_sample.zarr"))

# Build empty zarr plate scaffold.
plate = build_zarr_scaffold(root_dir=zarr_root,
                            files=files,
                            name='CV8000_sample',
                            layout=96,
                            order_name="example-order",
                            barcode="example-barcode")

In [9]:
channels = ['1', '2']
# Add image data to wells
for well in tqdm(files['well'].unique()):
    well_files = files[files['well'] == well]
    
    img, hists, ch_metadata, metadata, roi_tables = get_well_image_CZYX(
        well_files=well_files,
        channel_metadata_source=channel_metadata,
        channels=channels,
    )
    
    well_group = plate[well[0]][str(int(well[1:]))][0]
    write_czyx_image_to_well(img, hists, ch_metadata, metadata, well_group)
    
    # Write all ROI tables
    for roi_table in roi_tables:
        write_roi_table(roi_tables[roi_table], roi_table, well_group)

  0%|          | 0/3 [00:00<?, ?it/s]